In [1]:
from sqlalchemy.ext.asyncio import create_async_engine
import os
from dotenv import load_dotenv

_ = load_dotenv()

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST", "localhost")
database = os.getenv("DB_NAME")

# Create the connection string
connection_string = f"mysql+aiomysql://{username}:{password}@{host}/{database}"

# Create an engine instance
engine = create_async_engine(connection_string)

try:
    # Test the connection
    async with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Error connecting to database: {e}")

Connection successful!
